In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import requests
from pathlib import Path
import pandas

In [2]:
def calculate_accuracy(model_outputs, labels):
    _, predicted = torch.max(model_outputs, dim=1)
    correct = (predicted == labels).sum().item()
    return 100 * (correct / len(labels))

# Getting a multiclass dataset

In [3]:
DATA_PATH = 'penguins.csv'

In [4]:
if Path(DATA_PATH).is_file():
    print(f"{DATA_PATH} already exists, skipping download.")
else:
    print(f"Downloading data...")
    req = requests.get("https://gist.githubusercontent.com/slopp/ce3b90b9168f2f921784de84fa445651/raw/4ecf3041f0ed4913e7c230758733948bc561f434/penguins.csv")
    with open(DATA_PATH, "wb") as f:
        f.write(req.content)
    print("Done")

penguins.csv already exists, skipping download.


In [5]:
df = pandas.read_csv(DATA_PATH)

## Drop rows with NaN

In [6]:
df = df.dropna()

In [7]:
df = df[["bill_length_mm", "bill_depth_mm","flipper_length_mm","body_mass_g","species"]]

## Show unique classes (species)

In [8]:
df["species"].unique()

array(['Adelie', 'Gentoo', 'Chinstrap'], dtype=object)

## Map unique classes to numerical value

In [9]:
class_dict = {}
for idx, val in enumerate(df["species"].unique()):
    class_dict[val] = float(idx)
df["species"] = df["species"].map(class_dict)

## Split into data and target

In [10]:
X_df = df.iloc[:,:-1]
y_df = df.iloc[:,-1:]

## Get device for device-agnostic code

In [11]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'cuda'

## Create tensors

In [12]:
X = torch.tensor(X_df.values, dtype=torch.float32)
y = torch.tensor(y_df.values).squeeze()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
X_train = X_train.to(device).type(torch.float)
X_test = X_test.to(device).type(torch.float)
y_train = y_train.to(device).type(torch.int64)
y_test= y_test.to(device).type(torch.int64)

## Build model

In [14]:
class PenguinModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features)
        )
    def forward(self, x):
        return self.linear_layer_stack(x)

In [15]:
model = PenguinModel(input_features=4, output_features=3, hidden_units=32).to(device)

## Setup loss function and optimizer

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Train and Eval loop

In [17]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([266, 4]),
 torch.Size([266]),
 torch.Size([67, 4]),
 torch.Size([67]))

In [18]:
epochs = 5000

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    y_logits = model(X_train)
    
    loss = loss_fn(y_logits, y_train)
    acc = calculate_accuracy(y_logits, y_train)
    loss.backward()
    optimizer.step()
    
    model.eval()
    with torch.inference_mode():
        test_logits = model(X_test)
        test_loss = loss_fn(test_logits, y_test)
        test_acc = calculate_accuracy(test_logits, y_test)
    if (epoch % 100 == 0) or (epoch == epochs-1):
        print(f"Epoch: {epoch} | Loss: {loss:.4f}, Acc: {acc:.2f}% | Test loss: {test_loss:.4f}, Test acc: {test_acc:.2f}%") 

Epoch: 0 | Loss: 81.8743, Acc: 20.30% | Test loss: 64.0417, Test acc: 20.90%
Epoch: 100 | Loss: 0.7304, Acc: 66.92% | Test loss: 0.7611, Test acc: 67.16%
Epoch: 200 | Loss: 0.6961, Acc: 67.29% | Test loss: 0.7338, Test acc: 67.16%
Epoch: 300 | Loss: 0.6635, Acc: 68.42% | Test loss: 0.7041, Test acc: 68.66%
Epoch: 400 | Loss: 0.6293, Acc: 69.55% | Test loss: 0.6730, Test acc: 70.15%
Epoch: 500 | Loss: 0.5942, Acc: 70.30% | Test loss: 0.6407, Test acc: 71.64%
Epoch: 600 | Loss: 0.5759, Acc: 74.81% | Test loss: 0.6343, Test acc: 68.66%
Epoch: 700 | Loss: 0.5262, Acc: 74.81% | Test loss: 0.5776, Test acc: 76.12%
Epoch: 800 | Loss: 0.4938, Acc: 75.94% | Test loss: 0.5495, Test acc: 82.09%
Epoch: 900 | Loss: 0.4648, Acc: 83.83% | Test loss: 0.5202, Test acc: 74.63%
Epoch: 1000 | Loss: 0.4298, Acc: 81.20% | Test loss: 0.4884, Test acc: 89.55%
Epoch: 1100 | Loss: 0.3940, Acc: 86.84% | Test loss: 0.4529, Test acc: 89.55%
Epoch: 1200 | Loss: 0.3708, Acc: 93.98% | Test loss: 0.4288, Test acc: 80.

## Evaluate

In [19]:
from torchmetrics import Accuracy

In [20]:
logits = model(X.to(device))
softmax = torch.softmax(logits, dim=1)
labels = torch.argmax(softmax, dim= 1)

In [21]:
torchmetric_acc = Accuracy(task='multiclass', num_classes=3).to(device)

In [22]:
y.type(torch.int)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [43]:
model_acc = torchmetric_acc(labels.to(device), y.type(torch.int).to(device))
model_acc = model_acc.item() * 100

In [48]:
print(f"Model Accuracy: {model_acc:.2f}%")

Model Accuracy: 98.80%
